# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.0.1


In [2]:
%pip show jax | grep Version

Version: 0.4.4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wildsonbbl (wildson). Use `wandb login --relogin` to force relogin


True

In [4]:
!nvidia-smi

Tue Jul  4 00:40:36 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.06              Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GT 1030         On  | 00000000:01:00.0  On |                  N/A |
| 57%   38C    P0              N/A /  30W |    378MiB /  2048MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## PNA EPCSAFT

In [9]:
!python main.py --workdir=./training --config=configs/default.py \
--config.num_train_steps={int(4000)} --config.log_every_steps=14 --config.pad_size=64\
--config.checkpoint_every_steps=2000 --config.learning_rate=0.001 --config.warmup_steps=14 --config.optimizer={'adam'} --config.batch_size=128\
--config.propagation_depth={int(3)} --config.hidden_dim={int(64)} --config.num_mlp_layers={int(1)} --config.num_para={int(3)} --config.pre_layers=1 --config.post_layers=1

x64 on
I0704 00:53:33.436789 140522211104576 main.py:26] JAX host: 0 / 1
I0704 00:53:33.437009 140522211104576 main.py:27] JAX local devices: [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
I0704 00:53:33.437403 140522211104576 main.py:29] Calling train and evaluate
wandb: Currently logged in as: wildsonbbl (wildson). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.2
wandb: Run data is saved locally in /home/wildsonbbl/documents/code/ePC-SAFT/wandb/run-20230704_005338-rxii8p5a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lively-eon-392
wandb: ⭐️ View project at https://wandb.ai/wildson/gnn-pc-saft
wandb: 🚀 View run at https://wandb.ai/wildson/gnn-pc-saft/runs/rxii8p5a
I0704 00:53:47.061941 140522211104576 train.py:88] Obtaining datasets.
inchis saved: 114
I0704 00:53:47.161800 140522211104576 train.py:114